In [ ]:
# import library
import nltk
from nltk.corpus import stopwords
from random import randint
from collections import Counter 
import ast

In [ ]:
# Buat tuple yang menampung lirik lagu dan sentimen
def create_tuple( fileName ):
    open_file = open(fileName)
    read_file = open_file.read()
    collection = ast.literal_eval(read_file)
    result = []
    for i in collection:
        record = (i["lyric"], i["sentiment"])
        result.append(record)
    return result

In [ ]:
# Fungsi untuk menghapus stopwords dari lirik lagu dalam training set
def stop_words_removal(lirik):
    array = []
    for i in lirik:
        sentimen = i[1]
        stop = set(stopwords.words('english')) 
        lirik2 = [word for word in i[0].split() if word not in stop]
        combine = (lirik2 , sentimen)
        array.append(combine)
    return array 

In [ ]:
# Fungsi untuk return semua words dalam dataset lirik lagu
def get_kata(lirik):
    keseluruhan_kata = []
    for(words, sentimen) in lirik:
        keseluruhan_kata.extend(words)
    return keseluruhan_kata

In [ ]:
# Fungsi untuk menemukan frekuensi distribusi dari seluruh kata
def fitur_kata(daftar_kata):
	  daftar_kata = nltk.FreqDist(daftar_kata)
	  fitur_kata = daftar_kata.keys()
	  return fitur_kata

In [ ]:
# Fungsi untuk mengekstrasi fitur dari kata
def extract_features(doc):
	doc_words = set(doc)
	features = {}
	for word in fitur_kata: 
		features['contains(%s)' %word] = (word in doc_words)
	return features 

In [ ]:
# Fungsi untuk menghapus stopwords dari history lirik
def stop_word_removal1(lirik):
    stop = set(stopwords.words('english'))
    lirik2 = [word for word in lirik.split() if word not in stop]
    return lirik2

In [ ]:
# Fungsi untuk return daftar berdasarkan history milik user
def user_sentiment(file):
    open_file = open(file)
    read_file = open_file.read()
    split_file= read_file.split("|")
    hasil=[]
    for lirik in split_file:
        lirik2=stop_word_removal1(lirik)
        output = classifier.classify(extract_features(lirik2))
        hasil.append(output)
    return hasil

In [ ]:
# Fungsi untuk merekomendasikan lagu selanjutnya kepada user berdasarkan history user
def recommendedSong(history):
    c= Counter(history)
    open_file = open("training_original.txt")
    read_file = open_file.read()
    collection = ast.literal_eval(read_file)
    if c['P']>=c['N']:
        songs_positive=[i for i in collection if i["sentiment"]=='P']
        return songs_positive[randint(0,len(songs_positive)-1)]["name"]
    else:
        songs_negative=[i for i in collection if i["sentiment"]=='N']
        return songs_negative[randint(0,len(songs_negative)-1)]["name"]

In [ ]:
# Training model menggunakan training set

import nltk
nltk.download('stopwords')

# creating tuples
# give path of file "training.txt"
lirik = create_tuple("training_original.txt")
# removing stopwords
corpus2 = stop_words_removal(lirik)
# Extracting Features
fitur_kata = fitur_kata(get_kata(corpus2))
# Applying Features
training_set = nltk.classify.apply_features(extract_features,corpus2)
# Training Classifier
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [ ]:
# Testing model dengan menggunakan dataset test

lirik1= create_tuple("testing_original.txt")
# removing stopwords
corpus1 = stop_words_removal(lirik1)
# applying classifier model on test dataset
test_set = nltk.classify.apply_features(extract_features,corpus1)


In [ ]:
# Cek Akurasi
print ("\t" + "Akurasi model:" + str(nltk.classify.accuracy(classifier, test_set)))

In [ ]:
#Test model prediksi sentiment berdasarkan 1 lirik
def sentiment(output):
    if str(output) == "P":
        return "Sentimen Positif"
    if str(output) == "N":
        return "Sentimen Negatif"
    if str(output) == "A":
        return "Sentiment Tidak Terdefinisi"

        
lirik = "Gotta take my chance tonight"
output = classifier.classify(extract_features(lirik.split()))
print (sentiment(output))

In [ ]:
#  Gunakan model untuk merekomendasikan lagu
open_file = open("User_Last_5_Songs.txt")
read_file = open_file.read()
split_file=read_file.split("|")
result_list=[]

daftar_terakhir= user_sentiment("User_Last_5_Songs.txt") 
print ("recommend song: " + recommendedSong(daftar_terakhir))